In [ ]:
%useLatestDescriptors
%use dataframe
%use kandy
%use ktor-client
%use spark

@file:DependsOn("org.jetbrains.kotlinx:kotlinx-io-core:0.8.0")
@file:DependsOn("org.apache.commons:commons-compress:1.28.0")

In [ ]:
import kotlinx.io.asInputStream
import kotlinx.io.buffered
import kotlinx.io.files.Path
import kotlinx.io.files.SystemFileSystem
import org.apache.commons.compress.archivers.tar.TarArchiveInputStream
import org.apache.commons.compress.compressors.gzip.GzipCompressorInputStream

private val remoteDataUrl = "https://github.com/ageron/handson-ml/raw/master/datasets"
private val housingUrl = "$remoteDataUrl/housing/housing.tgz"

private val localDataDir = "/Users/markn/IdeaProjects/ml-project/datasets"
private val housingTgzFile = "housing.tgz"
private val housingCsvFile = "housing.csv"

val housingTgzPath = Path("$localDataDir/$housingTgzFile")
val housingCsvPath = Path("$localDataDir/$housingCsvFile")

fun loadHousingTgz() : Path {
    val data = http.get(housingUrl).readBytes()
    SystemFileSystem.createDirectories(Path(localDataDir))
    SystemFileSystem.sink(housingTgzPath).buffered().use {
        it.write(data)
    }
    return housingTgzPath
}

fun extractHousingCsv(path: Path) : Path {
    SystemFileSystem.source(path).buffered().use { source ->
        GzipCompressorInputStream(source.asInputStream()).use { gins ->
            TarArchiveInputStream(gins).use { tais ->
                var entry = tais.nextEntry
                while (entry != null) {
                    if (entry.name == housingCsvFile) {
                        SystemFileSystem.sink(housingCsvPath).buffered().use { sink ->
                            sink.write(tais.readBytes())
                        }
                    }
                    entry = tais.nextEntry
                }
            }
        }
    }
    return housingCsvPath
}

In [ ]:
val housingTgz = loadHousingTgz()
extractHousingCsv(housingTgz)

In [ ]:
val housing = DataFrame.readCsv(File(housingCsvPath.toString()))

In [ ]:
val plots = listOf(
    housing.plot { histogram(longitude) },
    housing.plot { histogram(latitude) },
    housing.plot { histogram(housing_median_age) },

    housing.plot { histogram(total_rooms) },
    housing.plot { histogram(total_bedrooms.map { it ?: -1.0 }.filter { it != -1.0 }) },
    housing.plot { histogram(population) },

    housing.plot { histogram(households) },
    housing.plot { histogram(median_income) },
    housing.plot { histogram(median_house_value) },
)
plotGrid(plots, nCol = 3)

In [66]:
import kotlin.random.Random

var housingTrain = housing
    .shuffle(Random(42)).take((housing.rowsCount() * 0.8).roundToInt())
    .add("income_cat") { ceil(median_income / 1.5).coerceIn(minimumValue = 1.0, maximumValue = 5.0) }
val housingTest = housing.shuffle(Random(42)).take((housing.rowsCount() * 0.2).roundToInt())

In [67]:
housingTrain.plot {
    histogram("income_cat")
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="aNDN2M"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"income_cat",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"count"
},
"stat":"identity",
"data":{
"count":[664.0,0.0,0.0,0.0,0.0,5272.0,0.0,0.0,0.0,0.0,5778.0,0.0,0.0,0.0,0.0,2872.0,0.0,0.0,0.0,1926.0],
"x":[1.1,1.2999999999999998,1.5,1.7000000000000002,1.9,2.1,2.3000000000000003,2.5,2.7,2.9,3.1,3.3000000000000003,3.5,3.7,3.9000000000000004,4.1,4.300000000000001,4.5,4.7,4.9]
},
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"bar",
"data_meta":{
"series_annotations":[{
"type":"float",
"column":"x"
},{
"type":"int",
"column":"count"
}]
}
}],
"spec_id":"98"
};
 var containerDiv = document.getElementById("aNDN2M");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.0 
 
 
 
 
 
 
 
 
 1.5 
 
 
 
 
 
 
 
 
 2.0 
 
 
 
 
 
 
 
 
 2.5 
 
 
 
 
 
 
 
 
 3.0 
 
 
 
 
 
 
 
 
 3.5 
 
 
 
 
 
 
 
 
 4.0 
 
 
 
 
 
 
 
 
 4.5 
 
 
 
 
 
 
 
 
 5.0 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 1,000 
 
 
 
 
 
 
 2,000 
 
 
 
 
 
 
 3,000 
 
 
 
 
 
 
 4,000 
 
 
 
 
 
 
 5,000 
 
 
 
 
 
 
 6,000 
 
 
 
 
 
 
 
 
 count 
 
 
 
 
 income_cat